In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/tmp/ipykernel_29362/912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ExampleApp") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()
# Set the log level to ERROR to suppress WARN messages
spark.sparkContext.setLogLevel("ERROR")


24/06/23 00:21:06 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.1.20 instead (on interface wlp61s0)
24/06/23 00:21:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/23 00:21:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.csv("/home/pk/DAEN690/package_data/PKG_Origin_PcScanLevel.txt", header=True, sep="|")
df.show()

+-----------+---------------+------------+--------------------+------------+--------------------+
|MailPieceID|TrackingEventID|EventZIPCode|   PTSEventTimestamp|PTSEventCode|        PTSEventDesc|
+-----------+---------------+------------+--------------------+------------+--------------------+
|88266058993|        1000032|       21904|1/19/2024 12:26:3...|          VC|PACKAGE RESEARCH ...|
|88251887847|        1000011|       24501|1/20/2024 09:04:1...|          VC|PACKAGE RESEARCH ...|
|88310099929|        1000028|       33605|1/20/2024 10:30:2...|          LX| LOOP MAIL EXCEPTION|
|88172867268|        1000007|       42104|1/9/2024 18:03:12...|          TM| SHIPMENT ACCEPTANCE|
|88173640884|        1000044|       80209|1/16/2024 15:02:3...|          VC|PACKAGE RESEARCH ...|
|88371164514|        1000025|       38004|1/24/2024 06:50:1...|          VC|PACKAGE RESEARCH ...|
|88247057520|        1000036|       45215|1/17/2024 07:35:0...|          VC|PACKAGE RESEARCH ...|
|88275236075|       

In [6]:
df.printSchema()

root
 |-- MailPieceID: string (nullable = true)
 |-- TrackingEventID: string (nullable = true)
 |-- EventZIPCode: string (nullable = true)
 |-- PTSEventTimestamp: string (nullable = true)
 |-- PTSEventCode: string (nullable = true)
 |-- PTSEventDesc: string (nullable = true)



In [7]:
from pyspark.sql.functions import to_timestamp
date_time_fields = ["PTSEventTimestamp"]
df = df.withColumn("PTSEventTimestamp", to_timestamp("PTSEventTimestamp", "M/d/yyyy HH:mm:ss"))    
df.printSchema()

root
 |-- MailPieceID: string (nullable = true)
 |-- TrackingEventID: string (nullable = true)
 |-- EventZIPCode: string (nullable = true)
 |-- PTSEventTimestamp: timestamp (nullable = true)
 |-- PTSEventCode: string (nullable = true)
 |-- PTSEventDesc: string (nullable = true)



In [8]:
df.createOrReplaceTempView("scans")

result_df = spark.sql("""
    SELECT
        MailPieceID,
        MIN(PTSEventTimestamp) AS min_scan_datetime,
        MAX(PTSEventTimestamp) AS max_scan_datetime,
        COUNT(*) AS total_scans,
        COUNT (DISTINCT EventZIPCode) AS Distinct_zip_scans,
        count(distinct PTSEventDesc) as Distinct_event_scans,
        (DATEDIFF(MAX(PTSEventTimestamp), MIN(PTSEventTimestamp)) * 24 * 60) AS time_delta_minutes
    FROM
        scans
    GROUP BY
        MailPieceID
  
""")

result_df.show()


+-----------+-------------------+-------------------+-----------+------------------+--------------------+------------------+
|MailPieceID|  min_scan_datetime|  max_scan_datetime|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|
+-----------+-------------------+-------------------+-----------+------------------+--------------------+------------------+
|88143492237|2024-01-08 16:08:32|2024-01-12 15:55:13|          7|                 3|                   7|              5760|
|88145979427|2024-01-08 09:35:29|2024-01-12 10:58:00|          7|                 3|                   7|              5760|
|88179517768|2024-01-10 04:45:56|2024-01-13 15:33:20|          3|                 3|                   3|              4320|
|88071008889|2024-01-04 20:07:34|2024-01-10 16:46:30|          6|                 4|                   6|              8640|
|88151819148|2024-01-08 13:09:54|2024-01-11 11:08:00|          7|                 5|                   6|              4320|


In [14]:
result_df.printSchema()

root
 |-- MailPieceID: string (nullable = true)
 |-- min_scan_datetime: timestamp (nullable = true)
 |-- max_scan_datetime: timestamp (nullable = true)
 |-- total_scans: long (nullable = false)
 |-- Distinct_zip_scans: long (nullable = false)
 |-- Distinct_event_scans: long (nullable = false)
 |-- time_delta_minutes: integer (nullable = true)



In [15]:
# Save the result as a Parquet file
result_df.write.mode("overwrite").parquet("/home/pk/DAEN690/mail_data/Scan Output/result.parquet")

24/06/20 12:04:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/20 12:04:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/20 12:04:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/20 12:04:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/20 12:04:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/20 12:04:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/20 12:04:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/20 12:04:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/20 12:05:17 WARN RowBasedKeyValueBatch: Calling spill() on

In [9]:
# Stop the Spark session
spark.stop()
